In [1]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('winequality-red.csv')

In [3]:
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [4]:
X = df.iloc[:,1:].copy()

In [5]:
target = df['quality'].copy()

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,target,test_size=0.3,random_state=0)

In [7]:
for col in X_train.columns:
    X_train[col] = StandardScaler().fit_transform(X_train[col].values.reshape(-1,1))

/home/pramila/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/pramila/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/pramila/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [8]:
mu = np.mean(X_train,axis=0).values.reshape(11,1) # Mean vector mu --> Since the data has been standardized, the data means are zero 

In [9]:
mu

array([[-1.21439945e-16],
       [-2.62276547e-16],
       [ 2.11527747e-16],
       [-3.01615549e-17],
       [ 1.43862680e-16],
       [ 1.23027395e-17],
       [-5.91710178e-14],
       [ 1.00555052e-15],
       [-3.55191995e-17],
       [-8.83812931e-16],
       [-3.52215526e-16]])

In [10]:
mu_k = []

In [11]:
for i,orchid in enumerate(np.unique(df['quality'])):
    mu_k.append(np.mean(X_train.where(df['quality']==orchid),axis=0))

mu_k = np.array(mu_k).T

In [12]:
# Computing the Scatter within and Scatter between matrices

data_SW = []
Nc = []
for i,quality in enumerate(np.unique(df['quality'])):
    a = np.array(X_train.where(df['quality']==quality).dropna().values-mu_k[:,i].reshape(1,11))
    data_SW.append(np.dot(a.T,a))
    Nc.append(np.sum(df['quality']==quality))
SW = np.sum(data_SW,axis=0)

SB = np.dot(Nc*np.array(mu_k-mu),np.array(mu_k-mu).T)
   
# Computing the Eigenvalues and Eigenvectors of SW^-1 SB
eigval, eigvec = np.linalg.eig(np.dot(np.linalg.inv(SW),SB))


    
# Selecting the two largest eigenvalues 
eigen_pairs = [[np.abs(eigval[i]),eigvec[:,i]] for i in range(len(eigval))]
eigen_pairs = sorted(eigen_pairs,key=lambda k: k[0],reverse=True)
w = np.hstack((eigen_pairs[0][1][:,np.newaxis].real,eigen_pairs[1][1][:,np.newaxis].real)) # Select two largest


# Transform the data
new_X = X_train.dot(w)


In [14]:
new_X

,0,1
92,-0.791510,0.509427
1017,0.423442,1.575719
1447,-0.791510,-0.384783
838,1.638393,0.456165
40,-0.791510,0.708140
...,...,...
763,-0.791510,-0.310266
835,-0.791510,-0.359944
1216,0.423442,-0.361736
559,0.423442,0.135048


Now this can be further used in any preferrable model.